In [ ]:
! pip install tensorflow

In [ ]:
# ==========================================================
# 🎯 INTRODUCTION — Deep Learning with Images
# ==========================================================
# In this notebook, we’ll understand how Deep Neural Networks (DNN)
# and Convolutional Neural Networks (CNN) work with image data.

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


In [ ]:

# ==========================================================
# 🧠 STEP 1 — Load and Explore the CIFAR-10 Dataset
# ==========================================================
# CIFAR-10 contains 60,000 color images (32x32 pixels, 3 channels)
# across 10 classes like airplane, car, cat, etc.

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print("Training data shape:", X_train.shape)
print("Single image shape:", X_train[0].shape)
print("Test data shape:", X_test.shape)

# Class names for CIFAR-10
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# 🔍 Visualize one image from each class >> Defines names of the 10 classes (to display later).
unique_classes = {}
for i, label in enumerate(y_train):
    if label[0] not in unique_classes:
        unique_classes[label[0]] = i
    if len(unique_classes) == 10:
        break

# Finds one example image for each class (for visualization).
plt.figure(figsize=(12, 6))
for class_idx, img_idx in unique_classes.items():
    plt.subplot(2, 5, class_idx + 1)
    plt.imshow(X_train[img_idx])
    plt.title(class_names[class_idx])
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ==========================================================
# 🧩 STEP 2 — Deep Neural Network (DNN)
# ==========================================================
# A DNN expects 1D vectors as input. But images are 3D (W, H, Channels)
# So we must FLATTEN each image to 1D.

# Flatten each image: (32,32,3) → (3072,) (because DNNs only accept flat inputs). || X_train.shape[0] → number of samples (e.g. 50000) || -1 → "let NumPy automatically calculate this dimension"

X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Normalizes pixel values from [0,255] → [0,1] for faster training.
X_train_flat = X_train_flat / 255.0
X_test_flat = X_test_flat / 255.0

# Converts class labels (like 3, 5, 7) into one-hot encoded vectors  (e.g., 3 → [0,0,0,1,0,0,0,0,0,0]).

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# 🏗️ Build the DNN model
model_dnn = Sequential([

    Dense(512, activation='relu', input_shape=(X_train_flat.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model_dnn.summary()

# Compile and train
model_dnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_dnn = model_dnn.fit(X_train_flat, y_train_cat, epochs=10, batch_size=128,
                            validation_data=(X_test_flat, y_test_cat))

# Evaluate DNN performance
print("\n📊 DNN Evaluation:")
model_dnn.evaluate(X_test_flat, y_test_cat)

In [ ]:


# ==========================================================
#  STEP 3 — Convolutional Neural Network (CNN)
# ==========================================================
# Unlike DNNs, CNNs can directly handle 2D/3D image data.
# They learn spatial features (edges, shapes, textures) automatically.

# Reload and normalize image data  >> 0-255   0-1
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# 🏗️ Build CNN model
model_cnn = Sequential([
    # Extract local patterns using filters
    Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),

    # Flatten 3D feature maps → 1D vector for dense layers
    Flatten(),

    # Fully connected layers
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),

    # Output layer (10 classes)
    Dense(10, activation='softmax')
])

model_cnn.summary()

# Compile and train CNN
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_cnn = model_cnn.fit(X_train, y_train, epochs=5, batch_size=128,
                            validation_data=(X_test, y_test))

# Evaluate CNN performance
print("\n📊 CNN Evaluation:")
model_cnn.evaluate(X_test, y_test)

In [ ]:

# ==========================================================
# 🧮 STEP 5 — CNN on MNIST (Grayscale Example)
# ==========================================================
from tensorflow.keras.datasets import mnist

(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

# Add channel dimension (since MNIST is grayscale)
x_train_mnist = x_train_mnist.reshape(-1, 28, 28, 1) / 255.0
x_test_mnist = x_test_mnist.reshape(-1, 28, 28, 1) / 255.0

# One-hot encode labels
y_train_mnist = to_categorical(y_train_mnist, 10)
y_test_mnist = to_categorical(y_test_mnist, 10)

print("MNIST sample shape:", x_train_mnist[0].shape)

# Small CNN for MNIST
model_mnist = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model_mnist.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_mnist.fit(x_train_mnist, y_train_mnist, epochs=3, validation_data=(x_test_mnist, y_test_mnist))

# ==========================================================
# ✅ SUMMARY
# ==========================================================
# • DNN: Works with flat data (not ideal for images)
# • CNN: Best for image data — learns spatial and visual features
# • Pooling layers reduce size while keeping important info
# • Flatten connects CNN → Dense layers for classification


In [ ]:
# https://www.pinecone.io/learn/series/image-search/cnn/  cnn visualized
# https://saturncloud.io/blog/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way/   cnn architecture